<a href="https://colab.research.google.com/github/SardarAmmer/Report-Generation/blob/main/Report_Generation(MKT).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title Dealer Format C (Updated with Sales Office)
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Alignment
from openpyxl.utils import get_column_letter

# === Load and Clean Data ===
df = pd.read_excel('/content/1st QTR Complete 25-26.xlsx')
df.columns = df.columns.str.replace(r'\s+', '', regex=True)
df = df[df['DistChannelName'].str.strip().str.lower() == 'dealers']
df['BillNetValuePKR'] = pd.to_numeric(df['BillNetValuePKR'].astype(str).str.replace(',', ''), errors='coerce')

# === Group Data by Sales Office, City, Product Group, Product, Plant ===
grouped = df.groupby(
    ['SalesOfficeName', 'SalesGroupName', 'PlantName', 'MaterialGrp4Name', 'MaterialName'],
    as_index=False
).agg({
    'DeliveryQuantity': 'sum',
    'BillNetValuePKR': 'sum'
})

# === Plant Order ===
plant_order = ['JB', 'Wah', 'DG Khan', 'Niz']

# === Create workbook ===
wb = Workbook()
ws = wb.active
ws.title = "Dealerwise Regional sales Analysis"

# === Header ===
header = ['Sales Office Name', 'City Name', 'Product Group', 'Product']
for plant in plant_order:
    header += [f'{plant} - Total Qty', f'{plant} - Total Amount']
ws.append(header)

# === Format Data ===
offices = grouped['SalesOfficeName'].dropna().unique()

for office in offices:
    office_data = grouped[grouped['SalesOfficeName'] == office]
    cities = office_data['SalesGroupName'].dropna().unique()

    for city in cities:
        city_data = office_data[office_data['SalesGroupName'] == city]

        # Get all unique product group/product combinations
        combos = city_data.groupby(['MaterialGrp4Name', 'MaterialName']).size().reset_index().values.tolist()

        for group, product, _ in combos:
            row = [office, city, group, product]
            for plant in plant_order:
                match = city_data[
                    (city_data['PlantName'] == plant) &
                    (city_data['MaterialGrp4Name'] == group) &
                    (city_data['MaterialName'] == product)
                ]
                if not match.empty:
                   qty = match['DeliveryQuantity'].values[0]
                   amt = match['BillNetValuePKR'].values[0]
                   qty = qty if pd.notna(qty) else "0"
                   amt = amt if pd.notna(amt) else "0"
                   row += [qty, amt]
                else:
                    row += ["nill", "nill"]

            ws.append(row)
            office = ""
            city = ""

# === Format Columns ===
for col in range(1, ws.max_column + 1):
    ws.column_dimensions[get_column_letter(col)].width = 18

for row in ws.iter_rows(min_row=2):
    for cell in row:
        cell.alignment = Alignment(wrap_text=True, vertical='top')

# === Save File ===

wb.save('/content/Dealerwise Regional Analysis Report(oct) xlsx')
print("✅ Saved as Updated_Citywise_Report_FormatA.xlsx")


KeyError: 'DistChannelName'